In [ ]:
import numpy as np
import glob2
import datetime
from pathlib import Path
from tqdm.notebook import tqdm
import pickle
from matplotlib import pyplot as plt
from utils.detection.association_geodesic import squarize
import matplotlib as mpl
import matplotlib.dates as mdates
from scipy import stats
from utils.physics.geodesic.distance import distance_point_point

plt.style.use('classic')
mpl.rcParams.update({
    "font.size": 10,
    "axes.titlesize": 15,
    "axes.labelsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "legend.fontsize": 10,
    "figure.titlesize": 15,
    "font.family": "serif",
    "pdf.fonttype": 42,
    "ps.fonttype": 42
})
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
import math
from numpy.linalg import LinAlgError
import pandas as pd
from scipy.interpolate import RegularGridInterpolator

from utils.data_reading.sound_data.station import StationsCatalog
from utils.physics.sound_model.spherical_sound_model import GridSphericalSoundModel as GridSoundModel, MonthlyHomogeneousSphericalSoundModel as HomogeneousSoundModel
from utils.detection.association_geodesic import compute_candidates, update_valid_grid, update_results, load_detections, compute_grids

In [ ]:
STATIONS = StationsCatalog("/media/plerolland/akoustik/MAHY").filter_out_undated().filter_out_unlocated()

for s in STATIONS:
    detections = []

    drift_ppm = 0
    if "clock_drift_ppm" in s.other_kwargs:
        drift_ppm = s.other_kwargs["clock_drift_ppm"]
    else:
        print(f"No drift for {s}")

    with open(f"../../../../data/detection/TiSSNet_Pn_fixed/{s.dataset}/{s.dataset}_{s.name}.pkl", "rb") as f:
        while True:
            try:
                detections.append(pickle.load(f))
            except EOFError:
                break

    new_detections = []
    for date, p in tqdm(detections[0]):
        delta =  2 * drift_ppm * (date-s.date_start).total_seconds() * 1e-6
        new_detections.append((date - datetime.timedelta(seconds=delta), p))

    out_dir = f"../../../../data/detection/TiSSNet_Pn_corr_inverted/{s.dataset}"
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    with open(f"{out_dir}/{s.dataset}_{s.name}.pkl", "wb") as f:
        pickle.dump(np.array(new_detections), f)

In [ ]:
path = "../../../../data/MAHY/loc_3D/drifts.csv"

df = pd.read_csv(path)  # shape (19, 4)

fig, axs = plt.subplots(nrows=5, ncols=4, figsize=(12, 10))
axs = axs.flatten()
c = ["drift GPS (raw→corr)", "drift OBS (raw→OBS)", "drift (corr→OBS)", "drift (corr_invert→OBS)"]
for i, row in df.iterrows():
    if np.isnan(row["drift GPS (raw→corr)"]):
        continue
    else:
        v = [row[c[0]], row[c[1]], row[c[2]], row[c[3]]]
        axs[i].bar(range(4), v)
        axs[i].set_xticks(range(4))
        axs[i].set_xticklabels(["GPS", "rawOBS",'corrOBS','corrIOBS'])
        axs[i].set_title(row["station"])
        axs[i].set_ylim(-0.2,0.2)
plt.tight_layout()
plt.savefig(f"../../../../data/MAHY/figures/dérives/drifts_measurements.png", dpi=500, bbox_inches='tight')